In [53]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/home/mrzaizai2k/code_Bao/LLM-with-RAG/notebook


In [54]:
from langchain_community.document_loaders import (
                                                  PyMuPDFLoader,
                                                  NewsURLLoader,
                                                  YoutubeLoader,
                                                  )
from langchain_text_splitters import RecursiveCharacterTextSplitter, TokenTextSplitter, NLTKTextSplitter, SpacyTextSplitter
import pymupdf

import os
import torch
import re
from PIL import Image
from natsort import natsorted
import shutil
from transformers import pipeline
from src.utils import *
from src.data_processing import MathLatexRecovery

In [55]:
device = take_device()

Selected device: cuda


In [56]:
data = config_parser(data_config_path = 'config/data_config.yaml')

In [57]:
# embeddings=HuggingFaceEmbeddings(model_name=model_config.get('embedding_model'),
#                                         model_kwargs={'device':take_device()})

In [58]:
pdf_path = "data/web_data/Lecture 3.pdf"
loader = PyMuPDFLoader(pdf_path)
ori_text = loader.load()
ori_text = combine_short_doc(ori_text, 100)

In [59]:
# for x in dir(loader):
#     if not x.startswith("_"):
#         print(x)

In [73]:
ori_text[:5]

[Document(page_content='1\nDATA ENGINEERING\nLECTURE 3\n QUERY PROCESSING \nOVERVIEW\nn Query to DBMS\nq Pre-processing\nq Optimization --> execution plan\nq Data access\nn Indexing\nn Merge sort\n2\n', metadata={'source': 'data/web_data/Lecture 3.pdf', 'file_path': 'data/web_data/Lecture 3.pdf', 'page': 1, 'total_pages': 92, 'format': 'PDF 1.7', 'title': 'Lecture 3.pptx', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Microsoft: Print To PDF', 'creationDate': "D:20240229170808+07'00'", 'modDate': "D:20240229170808+07'00'", 'trapped': ''}),
 Document(page_content='DATA STORAGE REVIEW\n3\n Disk storage devices (cont.)\n4\nJan - 2015\n Records\n5\nJan - 2015\n Blocking\n6\nJan - 2015\n', metadata={'source': 'data/web_data/Lecture 3.pdf', 'file_path': 'data/web_data/Lecture 3.pdf', 'page': 5, 'total_pages': 92, 'format': 'PDF 1.7', 'title': 'Lecture 3.pptx', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Microsoft: Print To PDF', 'creati

In [61]:
def is_latex_format(doc):
    # Check if doc.metadata exists and is a dictionary
    if not hasattr(doc, 'metadata') or not isinstance(doc.metadata, dict):
        return False
    
    # Check for "tex" in any metadata value (case-insensitive)
    return any("tex" in str(value).lower() for value in doc.metadata.values())


print(is_latex_format(ori_text[10]))           

False


In [62]:
text_list = [txt.page_content for txt in ori_text]

In [63]:
# Use a pipeline as a high-level helper

pipe = pipeline("text-classification", model="CrissWang/bert-math", torch_dtype=torch.float16, device = take_device())

Selected device: cuda


In [64]:
outputs = pipe(text_list, batch_size=8, truncation="only_first",)
print(outputs[:5])

[{'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}]


In [65]:
# for input, output in zip(text_list, outputs):
#     print(f"output: {output['label']} - Input: {input}")
#     print("---------------")

In [66]:
math_indices = [index for index, output in enumerate(outputs) if output['label'].lower() == "math"]

# Print the list of indices where the label is "math"
print(math_indices[:5])

[5, 6, 10, 12, 14]


In [67]:
print(len(text_list), len(math_indices))


74 26


In [68]:
doc = pymupdf.open(pdf_path)  # open document
tmp_path = "data/tmp/"
for page in doc:  # iterate through the pages
    if not (page.number in math_indices):
        continue
    pix = page.get_pixmap()  # render page to an image
    pix.save(f"{tmp_path}/%i.png" % page.number)  # store image as a PNG

In [69]:
nougat_processor = pipeline("image-to-text", model="facebook/nougat-small",  device = take_device(), max_new_tokens=1024 )

Selected device: cuda


In [70]:
def get_all_dir(root_dir, sort=True):
    # Get the list of items in the directory
    items = os.listdir(root_dir)
    # Sort the items list in natural order
    if sort:
        items = natsorted(items)
    # Create full paths
    image_list = [os.path.join(root_dir, item) for item in items]
    return image_list

image_list = get_all_dir(tmp_path)
print(image_list)

['data/tmp/5.png', 'data/tmp/6.png', 'data/tmp/10.png', 'data/tmp/12.png', 'data/tmp/14.png', 'data/tmp/17.png', 'data/tmp/21.png', 'data/tmp/22.png', 'data/tmp/23.png', 'data/tmp/24.png', 'data/tmp/29.png', 'data/tmp/30.png', 'data/tmp/32.png', 'data/tmp/33.png', 'data/tmp/34.png', 'data/tmp/41.png', 'data/tmp/42.png', 'data/tmp/52.png', 'data/tmp/64.png', 'data/tmp/65.png', 'data/tmp/66.png', 'data/tmp/67.png', 'data/tmp/69.png', 'data/tmp/70.png', 'data/tmp/71.png', 'data/tmp/72.png']


In [71]:
outputs = nougat_processor(image_list, batch_size=8, max_new_tokens=512,)

In [72]:
outputs[:5]

[[{'generated_text': 'Blocking\n\n(a) Block \\(i\\)Record 1Record 2Record 3Record 4Block \\(i+1\\)Record 4Record 5Record 6Record 6Block \\(i\\)Record 1Record 2Record 3Record 4Block \\(i+1\\)Record 4Record 5Record 6Record 7P\n\nJan - 2015'}],
 [{'generated_text': '.'}],
 [{'generated_text': 'EXAMPLE 1 (1/2)\n\n* An ordered file with \\(r\\) = 300,000 records\n* A disk with block size \\(B\\) = 4,096 bytes\n* File records are of fixed size and are unspanned, with record length \\(R\\) = 100 bytes\n* _bfr_ = \\(\\square\\) (\\(B\\)/\\(R\\)) = \\(\\square\\) (4,096/100) = 40 records per block\n* _b_ = \\(\\square\\) (\\(r\\)/\\(b\\)fr) = \\(\\square\\) (300,000/40) = 7,500 blocks\n* A b i n a r y se a r c h o n t h e d a t a file would need approximately \\(\\square\\) log2 \\(b\\) = (log27,500) = 13-block-accesses'}],
 [{'generated_text': 'Clustering Index\n* Defined on an ordered data file.\n* The data file is ordered on a _non-key field._\n* One index entry _each distinct value_ of the 

In [50]:
import shutil
def remove_and_recreate_folder(folder_path):
    # Check if the folder exists
    if os.path.exists(folder_path):
        # Remove the folder and all its contents
        shutil.rmtree(folder_path)
        print(f"Removed the folder: {folder_path}")
    
    # Recreate the folder
    os.makedirs(folder_path)
    print(f"Recreated the folder: {folder_path}")

remove_and_recreate_folder(tmp_path)

Removed the folder: data/tmp/
Recreated the folder: data/tmp/


In [51]:
for i, idx in enumerate(math_indices):
    ori_text[idx].page_content = outputs[i][0]["generated_text"]

In [52]:
ori_text[:5]

[Document(page_content='1\nDATA ENGINEERING\nLECTURE 3\n QUERY PROCESSING \nOVERVIEW\nn Query to DBMS\nq Pre-processing\nq Optimization --> execution plan\nq Data access\nn Indexing\nn Merge sort\n2\n', metadata={'source': 'data/web_data/Lecture 3.pdf', 'file_path': 'data/web_data/Lecture 3.pdf', 'page': 1, 'total_pages': 92, 'format': 'PDF 1.7', 'title': 'Lecture 3.pptx', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Microsoft: Print To PDF', 'creationDate': "D:20240229170808+07'00'", 'modDate': "D:20240229170808+07'00'", 'trapped': ''}),
 Document(page_content='DATA STORAGE REVIEW\n3\n Disk storage devices (cont.)\n4\nJan - 2015\n Records\n5\nJan - 2015\n Blocking\n6\nJan - 2015\n', metadata={'source': 'data/web_data/Lecture 3.pdf', 'file_path': 'data/web_data/Lecture 3.pdf', 'page': 5, 'total_pages': 92, 'format': 'PDF 1.7', 'title': 'Lecture 3.pptx', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Microsoft: Print To PDF', 'creati

In [38]:
post_processor = MathLatexRecovery()
outputs = post_processor.recover_math(ori_text)

Selected device: cuda
There are 20 / 38 pages has math


In [39]:
outputs[:5]

[Document(page_content='Discrete Random\nVariable\nNguyen Tien Thinh\nDiscrete Random\nVariable\nMultivariate Random\nVariable\nExpectation and\nVariance\nProbability Distribution\n(Draft).1\nChapter (Draft)\nDiscrete Random Variable\nFundamental Mathematics for Computer Science\nReferences:\n1. Ross, S. M. (2014). A first course in probability. Pearson.\n2. Kay, S. (2006). Intuitive probability and random processes using\nMATLAB®. Springer Science & Business Media.\nNguyen Tien Thinh\nFaculty of Computer Science and Engineering\nUniversity of Technology, VNU-HCM\n', metadata={'source': 'data/web_data/Random variable.pdf', 'file_path': 'data/web_data/Random variable.pdf', 'page': 0, 'total_pages': 38, 'format': 'PDF 1.5', 'title': '', 'author': 'Nguyen Tien Thinh', 'subject': 'Lecturer Fundamental Mathematics for Computer Science', 'keywords': '', 'creator': 'LaTeX with Beamer class', 'producer': 'pdfTeX-1.40.25', 'creationDate': "D:20231106094014+07'00'", 'modDate': "D:20231106094014+